# Import

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random
import copy

import scipy.stats as ss
from scipy.stats import norm
from scipy.stats import poisson
from scipy import spatial

# import code
from simulation import Simulation
from replicate_graph import layer_graph
import data
from data import select_dataset, get_station_g, set_random_speed_columns
from visualization import set_draw_attributes

import os

import re
from tqdm import tqdm

# Simulation

Select the appropriate dataset for the experiment you want to run. You will need to add random speed columns (assuming GIS free flow extraction) if not already present (uncomment ```set_random_speed_columns()```)

In [2]:
stations_df, distances_df = select_dataset("wcctci")
stations_df.head(1)

,Unnamed: 0,OID_,Name,TargetDestinationCount,SourceID,SourceOID,PosAlong,SideOfEdge,CurbApproach,Status,...,Cutoff_Minutes,Cutoff_TravelTime,Cutoff_Miles,Cutoff_Kilometers,Cutoff_TimeAt1KPH,Cutoff_WalkTime,Cutoff_TruckMinutes,Cutoff_TruckTravelTime,physical_capacity,charging_rate
0,0,1,MD/HD-0,NaN,1,7856914,0.987926,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,45


Intialize the simulation with basic parameters.

In [3]:
simulation_length = 24
battery_interval = 20
km_per_percent = 10
sim = Simulation(stations_df, distances_df, simulation_length, battery_interval, km_per_percent)

Add source and destination nodes, with custom or random distributions.

In [4]:
# Run this cell for custom distributions

# src_dstr = [10,20,30,40,50,60,50,40,30,20,30,40,50,60,50,40,30,20,10,5,0,0,0,0]
# sim.add_src("0",src_dstr)

# sim.add_dst("0",5)

In [5]:
# Run this cell for random (you can set average source volumes and stdevs, else default)

sim.random_srcs()
sim.random_dsts()

In [6]:
sim.run()

IndexError: list index out of range

In [ ]:
sim.data

In [ ]:
for v in sim.vehicle_list:
    print(v.path)

In [ ]:
for v in sim.vehicle_list:
    print(v.location, ":", v.distance_along_segment)

In [ ]:
sim.battery_G
labels = {node:node[-2:] if "in" in node else node[-3:] for node in list(sim.battery_G.nodes)}
pos, edge_colors = set_draw_attributes(sim.battery_G, station_G)
edge_labels = nx.get_edge_attributes(sim.battery_G,'weight')
nx.draw(sim.battery_G, pos = pos, labels = labels, edge_color = list(edge_colors.values()), 
        font_size = 8, font_color="white", node_color="black")
nx.draw_networkx_edge_labels(sim.battery_G, pos, edge_labels=edge_labels)

# Animation
TODO

# Create graphs from csv's

In [ ]:
stations_df.set_index("OID_").loc[228]["st_prv_cod"]

In [ ]:
distances_df.head(2)

In [ ]:
increment = 50
station_G = get_station_G()
G = layer_graph(station_G, increment)
G.nodes;

# Visualize general data structure

In [ ]:
# print(list(stations_df["OID_"]))
# print(station_G.nodes)
stations_df.head(1)

In [ ]:
# Set colors
for node in station_G.nodes:
    station_G.nodes[node]["color"] = "green"
    station_G.nodes[node]["node_size"] = 400
    
station_G.nodes["465"]["color"] = "blue" 
station_G.nodes["1"]["color"] = "blue" 
station_G.nodes["465"]["node_size"] = 400 
station_G.nodes["1"]["node_size"] = 400 

In [ ]:
pos=nx.get_node_attributes(station_G,'pos')
color = nx.get_node_attributes(station_G,'color').values()
node_size = list(nx.get_node_attributes(station_G,'node_size').values())
labels = {node:node for node in list(station_G.nodes)}
nx.draw(station_G, pos = pos, node_size=node_size, width=.5, node_color=color, labels = labels, 
        font_color = "white", font_size = 10)

In [ ]:
labels = {node:node[-2:] if "in" in node else node[-3:] for node in list(G.nodes)}
pos, edge_colors = set_draw_attributes(G, station_G)
edge_labels = nx.get_edge_attributes(G,'weight')
nx.draw(G, pos = pos, labels = labels, edge_color = list(edge_colors.values()), 
        font_size = 8, font_color="white", node_color="black")
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Experiments for Debugging

In [ ]:
stations_df = pd.read_csv("data/wcctci_stations-updated.csv")
distances_df = pd.read_csv("data/wcctci_distances.csv")
stations_df.columns
stations_df["longitude"] = stations_df["SnapX"]
stations_df["latitude"] = stations_df["SnapY"]

In [ ]:
columns = ['time', 'station', 'district', 'route', 'direction_of_travel', 
           'lane_type', 'station_length', 'samples', 'percent_observed', 
           'total_flow', 'avg_occupancy', 'avg_speed', 'delay_35', 'delay_40', 
           'delay_45', 'delay_50', 'delay_55', 'delay_60']
meta_csvs = {'3': 'd03_text_meta_2022_03_05.txt',
 '4':'d04_text_meta_2022_03_25.txt',
 '5':'d05_text_meta_2021_03_26.txt',
 '6':'d06_text_meta_2022_03_08.txt',
 '7':'d07_text_meta_2022_03_12.txt',
 '8':'d08_text_meta_2022_03_25.txt',
 '10':'d10_text_meta_2022_02_24.txt',
 '11':'d11_text_meta_2022_03_16.txt',
 '12':'d12_text_meta_2021_05_18.txt'}

df = pd.DataFrame()

for i in ['3', '4', '5', '6', '7', '8', '10', '11', '12']:
    num = i
    if int(i) < 10:
        num = "0" + i
    speed_df = speed_df = pd.read_csv("pems_ingest/station_data/d"+num+"_text_station_hour_2022_02.txt", sep = ',', header = None)
    speed_df = speed_df.iloc[: , :len(columns)]
    speed_df = speed_df.rename(columns = {i:columns[i] for i in range(len(columns))})
    speed_df["time"] = pd.to_datetime(speed_df['time'])
    speed_df["hour"] = speed_df["time"].dt.hour
    meta_df = pd.read_csv("pems_ingest/station_data/" + meta_csvs[i], sep = "\t")
    meta_df = meta_df[["ID", "Latitude", "Longitude"]]
    meta_df = meta_df.set_index("ID")
    speed_df = speed_df.join(meta_df, on = "station")
    df = pd.concat([df, speed_df])
df = df.dropna(axis="index", how="any", subset=["Latitude", "Longitude"])

In [ ]:
coord_distances_df = distances_df.join(stations_df.set_index("OID_")[["longitude", "latitude"]], on= "OriginID", rsuffix="_origin")
coord_distances_df = coord_distances_df.join(stations_df.set_index("OID_")[["longitude", "latitude"]], on= "DestinationID", rsuffix="_dst")

In [ ]:
def nearest_speed(x):
    lon = x["longitude"]
    lat = x["latitude"]
    d_lon = x["longitude_dst"]
    d_lat = x["latitude_dst"]
    src_res = tree.query([(lat,lon)])
    dst_res = tree.query([(d_lat,d_lon)])
    src_speed = df_h.iloc[src_res[1][0]]["avg_speed"]
    dst_speed = df_h.iloc[dst_res[1][0]]["avg_speed"]
    return (src_speed+dst_speed)/2

for h in tqdm(range(24)):
    df_h = df[df["hour"]==h]
    df_h = df_h.groupby(by=['station']).mean()
    df_h = df_h.dropna(axis="index", how="any", subset=["avg_speed", "Latitude", "Longitude"])
    coords = df_h[["Latitude", "Longitude"]].to_numpy()
    tree = spatial.KDTree(coords)
    coord_distances_df["speed_" + str(h)] = coord_distances_df.apply(nearest_speed, 1)

In [ ]:
coord_distances_df.to_csv("wcctci_coord_distances.csv")